In [ ]:
# Simple MLflow Model Registration - Notebook Ready
import mlflow
import mlflow.sklearn
import mlflow.pyfunc
from mlflow.tracking import MlflowClient
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from datetime import datetime

# =============================================================================
# CONFIGURATION - UPDATE THESE VALUES
# =============================================================================
MLFLOW_TRACKING_URI = "https://mlflow-server-199520990934.us-west1.run.app"  # Replace with your MLflow server
MODEL_NAME = "MyModel"  
EXPERIMENT_NAME = "model-serving-experiment"

# Set up MLflow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)

print(f"🔧 MLflow URI: {MLFLOW_TRACKING_URI}")
print(f"🔧 Model Name: {MODEL_NAME}")

# =============================================================================
# STEP 1: TRAIN AND REGISTER MODEL
# =============================================================================

def quick_model_registration():
    """Quick model training and registration"""
    
    with mlflow.start_run(run_name=f"quick_registration_{datetime.now().strftime('%H%M%S')}"):
        
        # 1. Create sample data (replace with your data)
        print("Creating sample data...")
        X, y = make_classification(n_samples=1000, n_features=20, random_state=42)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # 2. Train model (replace with your model)
        print("Training model...")
        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)
        
        # 3. Evaluate
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Accuracy: {accuracy:.4f}")
        
        # 4. Log parameters and metrics
        mlflow.log_param("model_type", "RandomForestClassifier")
        mlflow.log_param("n_estimators", 100)
        mlflow.log_metric("accuracy", accuracy)
        
        # 5. Register model
        print(f"Registering model as '{MODEL_NAME}'...")
        mlflow.sklearn.log_model(
            sk_model=model,
            artifact_path="model",
            registered_model_name=MODEL_NAME
        )
        
        run_id = mlflow.active_run().info.run_id
        print(f"✅ Model registered! Run ID: {run_id}")
        
        return run_id, model

# =============================================================================
# STEP 2: PROMOTE TO PRODUCTION
# =============================================================================

def promote_to_production(model_name, run_id=None):
    """Promote latest model version to Production"""
    
    client = MlflowClient()
    
    # Get the latest version
    try:
        if run_id:
            versions = client.search_model_versions(f"run_id='{run_id}'")
            version = versions[0].version
        else:
            latest_versions = client.get_latest_versions(model_name, stages=["None"])
            version = latest_versions[0].version
        
        print(f"Found version: {version}")
        
        # Transition to Production
        client.transition_model_version_stage(
            name=model_name,
            version=version,
            stage="Production"
        )
        
        print(f"Version {version} promoted to Production!")
        return version
        
    except Exception as e:
        print(f"Error promoting model: {e}")
        return None

# =============================================================================
# STEP 3: TEST LOADING
# =============================================================================

def test_model_loading(model_name, stage="Production"):
    """Test loading the registered model"""
    
    model_uri = f"models:/{model_name}/{stage}"
    print(f"Testing model loading from: {model_uri}")
    
    try:
        # Load model
        loaded_model = mlflow.pyfunc.load_model(model_uri)
        print("✅ Model loaded successfully!")
        
        # Test prediction with sample data
        test_data = np.random.rand(5, 20)  # 5 samples, 20 features
        predictions = loaded_model.predict(test_data)
        print(f"🔮 Sample predictions: {predictions}")
        
        return True
        
    except Exception as e:
        print(f"❌ Model loading failed: {e}")
        return False

# =============================================================================
# EXECUTION - RUN THIS SECTION
# =============================================================================

print("🚀 Starting model registration...")

# Step 1: Train and register
run_id, trained_model = quick_model_registration()

# Step 2: Promote to production
version = promote_to_production(MODEL_NAME, run_id)

# Step 3: Test loading
if version:
    success = test_model_loading(MODEL_NAME, "Production")
    
    if success:
        print(f"\n🎉 SUCCESS! Your model is ready for Cloud Run!")
        print(f"📋 Use this in your Cloud Run app:")
        print(f"    model = mlflow.pyfunc.load_model('models:/{MODEL_NAME}/Production')")
    else:
        print(f"\n⚠️ Model registered but loading failed. Check your MLflow server.")
else:
    print(f"\n❌ Failed to promote model to production.")

print(f"\n📊 MLflow UI: {MLFLOW_TRACKING_URI}")

2025/07/17 18:27:31 INFO mlflow.tracking.fluent: Experiment with name 'model-serving-experiment' does not exist. Creating a new experiment.


🔧 MLflow URI: https://mlflow-server-199520990934.us-west1.run.app
🔧 Model Name: MyModel
🚀 Starting model registration...
📊 Creating sample data...
🤖 Training model...
📈 Accuracy: 0.9000


2025/07/17 18:27:34 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


📦 Registering model as 'MyModel'...


2025/07/17 18:27:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'MyModel'.
2025/07/17 18:27:38 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: MyModel, version 1
Created version '1' of model 'MyModel'.


✅ Model registered! Run ID: 0902656cfcf2438fa79623f33582373c
🏃 View run quick_registration_182732 at: https://mlflow-server-199520990934.us-west1.run.app/#/experiments/1/runs/0902656cfcf2438fa79623f33582373c
🧪 View experiment at: https://mlflow-server-199520990934.us-west1.run.app/#/experiments/1
📋 Found version: 1
🚀 Version 1 promoted to Production!
🔄 Testing model loading from: models:/MyModel/Production


/var/folders/n_/3_x326jx1px01h5tp8dtcgzw0000gn/T/ipykernel_47375/946064162.py:91: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(

✅ Model loaded successfully!
🔮 Sample predictions: [1 1 1 1 1]

🎉 SUCCESS! Your model is ready for Cloud Run!
📋 Use this in your Cloud Run app:
    model = mlflow.pyfunc.load_model('models:/MyModel/Production')

📊 MLflow UI: https://mlflow-server-199520990934.us-west1.run.app


In [4]:
# Quick registration of an existing trained model
import mlflow
import mlflow.sklearn

# Assuming you already have a trained model called 'your_model'
mlflow.set_tracking_uri("https://mlflow-server-199520990934.us-west1.run.app")

# Train your model first
X, y = make_classification(n_samples=1000, n_features=20, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# This creates your_model
your_model = RandomForestClassifier(n_estimators=100, random_state=42)
your_model.fit(X_train, y_train)  # Now your_model is trained


with mlflow.start_run():
    mlflow.sklearn.log_model(
        sk_model=your_model,  # Your existing trained model
        artifact_path="model",
        registered_model_name="MyModel"
    )
    
    # Promote to production
    from mlflow.tracking import MlflowClient
    client = MlflowClient()
    latest_version = client.get_latest_versions("MyModel", stages=["None"])[0]
    client.transition_model_version_stage(
        name="MyModel",
        version=latest_version.version,
        stage="Production"
    )

print("✅ Model registered and promoted to Production!")

2025/07/17 18:27:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/17 18:27:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'MyModel' already exists. Creating a new version of this model...
2025/07/17 18:27:47 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: MyModel, version 2
Created version '2' of model 'MyModel'.
/var/folders/n_/3_x326jx1px01h5tp8dtcgzw0000gn/T/ipykernel_47375/2889998749.py:27: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-st

🏃 View run auspicious-trout-441 at: https://mlflow-server-199520990934.us-west1.run.app/#/experiments/1/runs/28de08846a5c4b3da66dadd6b843b9ee
🧪 View experiment at: https://mlflow-server-199520990934.us-west1.run.app/#/experiments/1
✅ Model registered and promoted to Production!
